In [1]:
from pathlib import Path
import math
import gdstk
import gdsfactory as gf
from qnlfactory import *
import numpy as np
from gdsfactory.cross_section import ComponentAlongPath
import qnlmodels.cpw as CPW
import qnlmodels.klopfenstein_taper as kt
from qnlfactory.components import (border, 
                                   launch, 
                                   solder_bump_array, 
                                   flip_chip_alignment,
                                   titanium_array)

layermap = QFaBLayers

In [2]:
# CONSTANTS
v0 = 3e8 #m/s Speed of light

In [10]:
#UTILS
def freq_band(line_delay, thru_length, phase_range):
    # v = thru_length / line_delay 
    # wavelen_bounds = 360 / phase_range[0] * length, 360 / phase_range[1] * length
    # freq_bounds = v / wavelen_bounds[0], v / wavelen_bounds[1]
    
    # Nomially equivalent
    freq_bounds =  phase_range[0] / 360 / line_delay, phase_range[1] / 360 / line_delay
    
    return freq_bounds

def length_band(freq_bounds, phases, group_velo):
    delay_bounds = phases[0] / 360 / freq_bounds[0], phases[1] / 360 / freq_bounds[1]
    length_bounds = delay_bounds[0]*group_velo, delay_bounds[1]*group_velo
    return length_bounds

def length_from_phase_shift(phase, max_freq, group_velo):
    min_wavelength = group_velo / max_freq
    min_length = min_wavelength * phase / 360
    return min_length

def phase_from_delay(freq, delay):
    return 360*delay*freq

In [55]:
print(length_from_phase_shift(30, 12e9, 0.5*v0)*1e2)

f_max = 13e9 #Hz = 1/s
f_min = 5e9
vg = 0.5*3e8 #m/s

print('min wavelength:', vg / f_max * 1e2, 'cm')
print('max wavelength:', vg / f_min * 1e2, 'cm')

print('10GHz wavelength ~', vg/10e9*1e2, 'cm')
print('10GHz wavelength (180 degrees out of phase) ~', 1.5*vg/10e9*1e2, 'cm')
print('10GHz wavelength (30 degrees out of phase) ~', (1+30/360)*vg/10e9*1e2, 'cm')
print('10GHz wavelength (150 degrees out of phase) ~', (1+150/360)*vg/10e9*1e2, 'cm')

print('7.5GHz wavelength ~', vg/7.5e9*1e2, 'cm')
print('7.5GHz wavelength (180 degrees out of phase) ~', 0.5*vg/7.5e9*1e2, 'cm')
print('7.5GHz wavelength (30 degrees out of phase) ~', (1-30/360)*vg/7.5e9*1e2, 'cm')
print('7.5GHz wavelength (150 degrees out of phase) ~', (1-150/360)*vg/7.5e9*1e2, 'cm')

center_freq = 7.5e9
phases = [30,60,90,120,150]
for phase in phases:
    center_wavelength = vg / center_freq #assumes symetric range
    out_phase_wavelength = center_wavelength*(phase/180)
    print(out_phase_wavelength*1e2, 'cm')

0.10416666666666667
min wavelength: 1.153846153846154 cm
max wavelength: 3.0 cm
10GHz wavelength ~ 1.5 cm
10GHz wavelength (180 degrees out of phase) ~ 2.25 cm
10GHz wavelength (30 degrees out of phase) ~ 1.625 cm
10GHz wavelength (150 degrees out of phase) ~ 2.125 cm
7.5GHz wavelength ~ 2.0 cm
7.5GHz wavelength (180 degrees out of phase) ~ 1.0 cm
7.5GHz wavelength (30 degrees out of phase) ~ 1.8333333333333333 cm
7.5GHz wavelength (150 degrees out of phase) ~ 1.1666666666666665 cm
0.3333333333333333 cm
0.6666666666666666 cm
1.0 cm
1.3333333333333333 cm
1.6666666666666667 cm


In [5]:
c = gf.Component()

line_width = 10
line_spacing = 6

bottom_chip_size=(10000,10000)
top_chip_size=(9000,9000)
N_lines = 4
margin = 900
start = (-top_chip_size[0]/2, top_chip_size[1]/2-margin)
end = (-top_chip_size[0]/2, -top_chip_size[1]/2+margin)

delta_phase = 60
phase_bounds = (delta_phase, 180-delta_phase) # degrees
fc = 7.5e9
gv = 0.4*v0
phases = np.linspace(start=phase_bounds[0], stop=phase_bounds[1], num=N_lines)
# phases = [40,65,105,130]
delta_ls = [length_from_phase_shift(phase, fc, gv) for phase in phases]
for dl in delta_ls: 
    print("dL = {:.2f} um".format(dl*1e6))
    print("L = {:.2f} cm".format(1+dl*1e2))


bottom_chip = border(bottom_chip_size)
bottom_ref = c << bottom_chip

top_chip = border(top_chip_size)
top_ref = c << top_chip


trl_trace = Trace(width=line_width, spacing=line_spacing)
launch_in = launch(connected_trace=trl_trace)
launch_out = launch(connected_trace=trl_trace, angle=180)

thru = gf.Component()

thru_cpw = Trace(width=line_width, spacing=line_spacing)
thru_cpw = thru_cpw.straight(top_chip_size[0]).make()
thru_text = gf.components.text('Thru', size=100, position=(-400, 300), layer=layermap.SC1_E)
thru << thru_cpw
thru << launch_in
thru << thru_text
thru_launch_out_ref = thru << launch_out
thru_launch_out_ref.dmovex(top_chip_size[0])
thru_ref = c.add_ref(thru, name='thru')
thru_ref.dmove(start)

separation_width = (top_chip_size[1]-2*margin)/(N_lines+1)
f=3
n_segments = 11
turn_radius = top_chip_size[0]*(f-2)/(2*f*(n_segments+2))

for i in range(len(delta_ls)):
    segment_length = 2*((f-2)*top_chip_size[0]/f + delta_ls[i]*1e6 - (n_segments+2)*np.pi*turn_radius + 2*turn_radius)/(n_segments+1) + 2*turn_radius
    line = gf.Component()
    line_cpw = Trace(width=line_width, spacing=line_spacing)
    line_cpw = line_cpw.straight(top_chip_size[0]/f).turn(turn_radius,-90).straight(segment_length/2-turn_radius)
    line_cpw = line_cpw.meander(num_segments=n_segments, length=segment_length, radius=turn_radius, turn=1)
    line_cpw = line_cpw.straight(segment_length/2-turn_radius).turn(turn_radius,90).straight(top_chip_size[0]/f).make()
    line_text = gf.components.text('Line{:.0f}'.format(i), size=100, position=(-400, 300), layer=layermap.SC1_E)
    line_degrees = gf.components.text('{:.0f}'.format(phases[i]), size=100, position=(-400, -400), layer=layermap.SC1_E)
    line << line_text
    line << line_degrees
    line << line_cpw
    line << launch_in
    line_launch_out_ref = line << launch_out
    line_launch_out_ref.dmovex(top_chip_size[0])
    line_ref = c.add_ref(line, name='Line_{:.0f}'.format(i+1))
    line_ref.dmove((-top_chip_size[0]/2, i*separation_width))
    line_ref.dmovey(-(N_lines-1)/2*separation_width)

metal_width = 100
array_spacing = ((4*turn_radius), separation_width/2)
n_bumps_wide = int((top_chip_size[0]-margin/2)/array_spacing[0])+1
n_bumps_tall = int((top_chip_size[1]-margin/2)/array_spacing[1])+1
sb_array = solder_bump_array(cols=n_bumps_wide, rows=n_bumps_tall, spacing=array_spacing, centered=True, underbump_metal_width=metal_width)
ti_array = titanium_array(cols=n_bumps_wide, rows=n_bumps_tall, spacing=array_spacing, centered=True, metal_width=metal_width)
fc_align = flip_chip_alignment(center_offset=(top_chip_size[0]-150-array_spacing[0], top_chip_size[1]-150-array_spacing[0]), shift=150)

sb_array_ref = c << sb_array
sb_array_ref.dmove((-array_spacing[0]/4,-array_spacing[1]/4))
ti_array_ref = c << ti_array
ti_array_ref.dmove((array_spacing[0]/4,array_spacing[1]/4))
fr_align_ref = c << fc_align

reflect = gf.Component()
reflect_line_separation = array_spacing[1]/2
r_cpw = Trace(width=line_width, spacing=line_spacing)
r_cpw = r_cpw.straight(top_chip_size[0]/2).make()
r_text = gf.components.text('Rflct', size=100, position=(-400, 300), layer=layermap.SC1_E)
reflect_in = gf.Component()
reflect_in << r_text
reflect_in << r_cpw
reflect_in << launch_in
reflect_out = gf.Component()
reflect_out << r_cpw
reflect_out.add_ref(launch_out).dmovex(top_chip_size[0]/2)
r_in_ref = reflect << reflect_in
r_out_ref = reflect << reflect_out
r_out_ref.dmovey(-reflect_line_separation).dmovex(top_chip_size[0]/2)

termination = gf.components.rectangle(size=(trl_trace.line_spacing, trl_trace.line_width+2*trl_trace.line_spacing), layer=layermap.SC1_E, centered=True)
reflect.add_ref(termination).dmove((top_chip_size[0]/2+trl_trace.line_spacing/2, 0))
reflect.add_ref(termination).dmove((top_chip_size[0]/2-trl_trace.line_spacing/2, -reflect_line_separation))
reflect_ref = c.add_ref(reflect, name='reflect')
reflect_ref.dmove((end))

c.show()

dL = 2666.67 um
L = 1.27 cm
dL = 3555.56 um
L = 1.36 cm
dL = 4444.44 um
L = 1.44 cm
dL = 5333.33 um
L = 1.53 cm
2024-12-11 20:42:34.380 | INFO     | kfactory.kcell:show:8727 - klive v0.3.3: Opened file '/Users/elias/Desktop/School/Quantum/AQT/build/gds/4095189768.oas'


In [7]:
l = 0.9e-2
v = 0.5*v0
wl = l 
f = v/wl
print(f*1e-9)

16.666666666666668
